In [19]:
#################################################################################
# In this program, we shall be developing a movie Rec System
#################################################################################


# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

!pip install findspark

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
#spark = SparkSession.builder.master("local[4]")  # this creates spark session to  run locally, the braket[] means number of cores 



In [20]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Rec System") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
movies = spark.read.csv("drive/My Drive/Colab Notebooks/Files/RecSys/movies.csv",header=True)
ratings = spark.read.csv("drive/My Drive/Colab Notebooks/Files/RecSys/ratings.csv",header=True)

In [23]:
ratings.show()


+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [24]:

ratings.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [25]:
from pyspark.sql import functions as F

ratings = ratings.\
    withColumn('userId', F.col('userId').cast('integer')).\
    withColumn('movieId', F.col('movieId').cast('integer')).\
    withColumn('rating', F.col('rating').cast('float')).\
    drop('timestamp')
ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
|     1|    163|   5.0|
|     1|    216|   5.0|
|     1|    223|   3.0|
|     1|    231|   5.0|
|     1|    235|   4.0|
|     1|    260|   5.0|
|     1|    296|   3.0|
|     1|    316|   3.0|
|     1|    333|   5.0|
|     1|    349|   4.0|
+------+-------+------+
only showing top 20 rows



In [26]:
# Sparsity shows how sparse is the matrix, e.g. 99% sparsity shows that, 99% of the entries in the matrix are missing.
# Count the total number of ratings in the dataset
numerator = ratings.select("rating").count()

# Count the number of distinct userIds and distinct movieIds
num_users = ratings.select("userId").distinct().count()
num_movies = ratings.select("movieId").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_movies

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  98.30% empty.


In [27]:
# Group data by userId, count ratings
userId_ratings = ratings.groupBy("userId").count().orderBy('count', ascending=False)
userId_ratings.show()

+------+-----+
|userId|count|
+------+-----+
|   414| 2698|
|   599| 2478|
|   474| 2108|
|   448| 1864|
|   274| 1346|
|   610| 1302|
|    68| 1260|
|   380| 1218|
|   606| 1115|
|   288| 1055|
|   249| 1046|
|   387| 1027|
|   182|  977|
|   307|  975|
|   603|  943|
|   298|  939|
|   177|  904|
|   318|  879|
|   232|  862|
|   480|  836|
+------+-----+
only showing top 20 rows



In [28]:

# Group data by movieId, count ratings
movieId_ratings = ratings.groupBy("movieId").count().orderBy('count', ascending=False)
movieId_ratings.show()

+-------+-----+
|movieId|count|
+-------+-----+
|    356|  329|
|    318|  317|
|    296|  307|
|    593|  279|
|   2571|  278|
|    260|  251|
|    480|  238|
|    110|  237|
|    589|  224|
|    527|  220|
|   2959|  218|
|      1|  215|
|   1196|  211|
|   2858|  204|
|     50|  204|
|     47|  203|
|    780|  202|
|    150|  201|
|   1198|  200|
|   4993|  198|
+-------+-----+
only showing top 20 rows



In [ ]:
##################################
# ALS MODEL --- Alternative Least Square Method 

In [29]:

# Import the required functions
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [30]:
# Split the ratings dataframe into training and test data
(train, test) = ratings.randomSplit([0.8, 0.2], seed=42)

# Set the ALS hyperparameters
als = ALS(
    userCol="userId", 
    itemCol="movieId", 
    ratingCol="rating", 
    rank =10, 
    maxIter =10, 
    regParam =.1,
    coldStartStrategy="drop", 
    nonnegative =True, 
    implicitPrefs = False
)

# Fit the model to the training_data
als_model = als.fit(train)

# Generate predictions on the test_data
test_predictions = als_model.transform(test)
test_predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   182|    471|   4.5| 3.0702088|
|   462|    471|   2.5|  3.492978|
|   610|    471|   4.0| 3.6468477|
|   176|    471|   5.0| 3.7472274|
|   448|    471|   4.0| 3.3893945|
|   608|    833|   0.5| 1.9699359|
|   463|   1088|   3.5| 4.0008287|
|   606|   1088|   3.0| 3.6406283|
|   563|   1088|   4.0| 3.2389886|
|   387|   1088|   1.5| 2.7619092|
|   381|   1088|   3.5| 3.7325559|
|   594|   1088|   4.5| 4.4393034|
|    10|   1088|   3.0| 4.0571632|
|   226|   1088|   1.0| 3.4202871|
|   414|   1088|   3.0| 3.3632245|
|    68|   1088|   3.5|  3.537528|
|   116|   1088|   4.5| 3.8114989|
|    42|   1088|   3.0| 3.4308286|
|   587|   1580|   4.0| 4.0509686|
|   577|   1580|   3.0| 3.1343348|
+------+-------+------+----------+
only showing top 20 rows



In [31]:
# Import RegressionEvaluator
from pyspark.ml.evaluation import RegressionEvaluator

# Complete the evaluator code
evaluator = RegressionEvaluator(metricName="rmse", labelCol="userId", predictionCol="rating")

# Extract the 3 parameters
print(evaluator.getMetricName())
print(evaluator.getLabelCol())
print(evaluator.getPredictionCol())


# Evaluate the "test_predictions" dataframe
RMSE = evaluator.evaluate(test_predictions)

# Print the RMSE
print ('RMSE = ', RMSE)

rmse
userId
rating
RMSE =  368.0588125025974


In [32]:
# Generate n Recommendations for all users
nrecommendations = als_model.recommendForAllUsers(10)
nrecommendations.limit(10).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   471|[[89904, 5.026805...|
|   463|[[5075, 5.2262783...|
|   496|[[25771, 5.000043...|
|   148|[[183897, 4.62388...|
|   540|[[96004, 5.202859...|
|   392|[[27251, 5.368526...|
|   243|[[5075, 6.142428]...|
|    31|[[183897, 5.64559...|
|   516|[[33649, 4.916319...|
|   580|[[74754, 5.603075...|
+------+--------------------+



In [33]:
#Explaining the recommendations 

from pyspark.sql.functions import split, explode

nrecommendations = nrecommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userId', F.col("rec_exp.movieId"), F.col("rec_exp.rating"))

nrecommendations.limit(10).show()

+------+-------+---------+
|userId|movieId|   rating|
+------+-------+---------+
|   471|  89904|5.0268054|
|   471|  25771| 5.011758|
|   471| 177593| 4.799852|
|   471|   3266| 4.793231|
|   471|  26326| 4.788419|
|   471|  58301|4.7722087|
|   471|   3224| 4.762935|
|   471|   5915|4.7489014|
|   471| 132333|4.7287564|
|   471|   4642|  4.71185|
+------+-------+---------+



In [34]:
#Explaining the recommendations 
nrecommendations.join(movies, on='movieId').filter('userId = 100').show()


+-------+------+---------+--------------------+--------------------+
|movieId|userId|   rating|               title|              genres|
+-------+------+---------+--------------------+--------------------+
|  26133|   100|  5.16446|Charlie Brown Chr...|Animation|Childre...|
|  33649|   100|5.0828953|  Saving Face (2004)|Comedy|Drama|Romance|
|  32892|   100| 5.063054|Ivan's Childhood ...|           Drama|War|
| 184245|   100| 5.031782|De platte jungle ...|         Documentary|
|  86237|   100| 5.031782|  Connections (1978)|         Documentary|
|  84273|   100| 5.031782|Zeitgeist: Moving...|         Documentary|
| 134796|   100| 5.031782|  Bitter Lake (2015)|         Documentary|
|   7071|   100| 5.031782|Woman Under the I...|               Drama|
| 117531|   100| 5.031782|    Watermark (2014)|         Documentary|
|  93988|   100|5.0140543|North & South (2004)|       Drama|Romance|
+-------+------+---------+--------------------+--------------------+



In [35]:
ratings.join(movies, on='movieId').filter('userId = 100').sort('rating', ascending=False).limit(10).show()


+-------+------+------+--------------------+--------------------+
|movieId|userId|rating|               title|              genres|
+-------+------+------+--------------------+--------------------+
|   1101|   100|   5.0|      Top Gun (1986)|      Action|Romance|
|   1958|   100|   5.0|Terms of Endearme...|        Comedy|Drama|
|   2423|   100|   5.0|Christmas Vacatio...|              Comedy|
|   4041|   100|   5.0|Officer and a Gen...|       Drama|Romance|
|   5620|   100|   5.0|Sweet Home Alabam...|      Comedy|Romance|
|    368|   100|   4.5|     Maverick (1994)|Adventure|Comedy|...|
|    934|   100|   4.5|Father of the Bri...|              Comedy|
|    539|   100|   4.5|Sleepless in Seat...|Comedy|Drama|Romance|
|     16|   100|   4.5|       Casino (1995)|         Crime|Drama|
|    553|   100|   4.5|    Tombstone (1993)|Action|Drama|Western|
+-------+------+------+--------------------+--------------------+

